In [1]:
import mygene
import numpy as np
import pandas as pd

from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests

### Example data for GSE90014 (human single-cell dataset of reprogrammed cells)

data_example: raw count matrix where row indices are gene names and column names are samples

The file example_vst.csv.gz is generated as a result of running the script VST_reprogramming.R

In [2]:
data = pd.read_csv("../output_reprogramming/example_vst.csv.gz", index_col=0)
data = data.T
scaler = StandardScaler()
data_z = pd.DataFrame(scaler.fit_transform(data), index=data.index, columns=data.columns)

### Metadata

metadata is manually curated and should include columns 'days_of_reprogramming' and 'cell_type'. In 'days_of_reprogramming', each sample should have numeric values, except for iPSC samples, which retain the string 'iPSC'.


In [3]:
import pandas as pd

metadata = pd.read_csv(
    "../data_example_reprogramming/example_metadata.csv", index_col=0
)

numeric_days = pd.to_numeric(metadata["days_of_reprogramming"], errors="coerce")
max_days = numeric_days.max()
metadata["days_of_reprogramming"] = (
    metadata["days_of_reprogramming"].replace("iPSC", max_days + 3).astype(int)
)

metadata

,cell_type,days_of_reprogramming
SRR5032364,Human dermal fibroblasts,0
SRR5032365,Human dermal fibroblasts,0
SRR5032366,Human dermal fibroblasts,0
SRR5032367,Human dermal fibroblasts,0
SRR5032368,Human dermal fibroblasts,0
...,...,...
SRR5032551,Human induced Pluripotent Stem cell,31
SRR5032552,Human induced Pluripotent Stem cell,31
SRR5032553,Human induced Pluripotent Stem cell,31
SRR5032554,Human induced Pluripotent Stem cell,31


### Hallmarks

In [4]:
senescence = pd.read_csv("../../../signatures/output_data_signatures/senescence_data.csv")
autophagy = pd.read_csv(
    "../../../signatures/output_data_signatures/autophagy_signatures.csv"
)
genomic = pd.read_csv(
    "../../../signatures/output_data_signatures/genomic_signatures_new.csv"
)
proteostasis = pd.read_csv(
    "../../../signatures/output_data_signatures/proteostasis_signatures.csv"
)
all_genes = pd.read_csv(
    "../../../signatures/output_data_signatures/all_hallmarks.csv", index_col=0
)

human_to_mouse = dict(zip(all_genes["ENSEMBL"], all_genes["ENSEMBL_mice"]))


def add_mouse_column(df):
    df["ENSEMBL_mice"] = df["ENSEMBL"].map(human_to_mouse)
    return df


senescence = add_mouse_column(senescence)
autophagy = add_mouse_column(autophagy)
genomic = add_mouse_column(genomic)
proteostasis = add_mouse_column(proteostasis)

In [5]:
senescence = senescence[["Gene", "Subprocess", "Hallmark", "ENSEMBL", "ENSEMBL_mice"]]
autophagy = autophagy[["Gene", "Subprocess", "Hallmark", "ENSEMBL", "ENSEMBL_mice"]]
genomic = genomic[["Gene", "Subprocess", "Hallmark", "ENSEMBL", "ENSEMBL_mice"]]
proteostasis = proteostasis[
    ["Gene", "Subprocess", "Hallmark", "ENSEMBL", "ENSEMBL_mice"]
]

all_hallmarks = pd.concat(
    [senescence, autophagy, genomic, proteostasis], ignore_index=True
)

### Computing Spearman correlation for each gene

In [6]:
def compute_hallmark_spearman(
    metadata: pd.DataFrame, expr_data: pd.DataFrame, all_hallmarks: pd.DataFrame
) -> pd.DataFrame:
    """
    Computes Spearman correlation for Hallmark genes across predefined reprogramming days.

    Assumes that `metadata['days_of_reprogramming']` already contains integer day values
    without any 'iPSC' entries.

    Parameters
    ----------
    metadata : pd.DataFrame
        Metadata DataFrame; its index matches that of `expr_data`, and the
        'days_of_reprogramming' column contains integer day values.
    expr_data : pd.DataFrame
        Expression matrix (rows correspond to samples, columns to gene Ensembl IDs).
    all_hallmarks : pd.DataFrame
        Hallmark annotations with columns 'ENSEMBL', 'Subprocess', and 'Hallmark'.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns
        ['Hallmark', 'Subprocess', 'GeneSymbol', 'SpearmanR', 'raw_pval'],
        sorted by Hallmark and Subprocess.
    """
    
    time_vec = metadata["days_of_reprogramming"]

    valid_hm = all_hallmarks[
        all_hallmarks["ENSEMBL"].isin(expr_data.columns)
    ].drop_duplicates(subset=["ENSEMBL"])

    records = []
    for ensembl_id in valid_hm["ENSEMBL"]:
        expr = expr_data[ensembl_id]
        if expr.nunique() <= 1:
            continue
        rho, pval = spearmanr(expr, time_vec, nan_policy="omit")
        records.append({"Gene": ensembl_id, "SpearmanR": rho, "raw_pval": pval})

    df_corr = pd.DataFrame(records)
    if df_corr.empty:
        cols = ["Hallmark", "Subprocess", "GeneSymbol", "SpearmanR", "raw_pval"]
        return pd.DataFrame(columns=cols)

    _, adj_pvals, _, signif = multipletests(df_corr["raw_pval"], method="fdr_bh")
    df_corr["FDR"] = adj_pvals
    df_corr["significant"] = signif

    annot = valid_hm[["ENSEMBL", "Subprocess", "Hallmark"]].drop_duplicates()
    df_corr = df_corr.merge(annot, left_on="Gene", right_on="ENSEMBL", how="left")

    mg = mygene.MyGeneInfo()
    ids = df_corr["Gene"].tolist()
    query_res = mg.querymany(
        ids, scopes="ensembl.gene", fields="symbol", species="human"
    )
    map_df = (
        pd.DataFrame(query_res)[["query", "symbol"]]
        .dropna()
        .rename(columns={"query": "Gene", "symbol": "GeneSymbol"})
    )
    df_corr = df_corr.merge(map_df, on="Gene", how="left")

    result = (
        df_corr[["Hallmark", "Subprocess", "GeneSymbol", "SpearmanR", "raw_pval"]]
        .sort_values(["Hallmark", "Subprocess"])
        .reset_index(drop=True)
    )
    return result

In [7]:
spearman_for_each_gene = compute_hallmark_spearman(metadata, data, all_hallmarks)
spearman_for_each_gene.head()

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


,Hallmark,Subprocess,GeneSymbol,SpearmanR,raw_pval
0,Autophagy,Autophagy core,ULK3,0.000395,0.995656
1,Autophagy,Autophagy core,TRAF6,0.029797,0.681602
2,Autophagy,Autophagy core,RB1CC1,-0.142026,0.049404
3,Autophagy,Autophagy core,ULK2,-0.147858,0.040690
4,Autophagy,Autophagy core,BECN1,-0.188781,0.008732


In [8]:
spearman_for_each_gene.to_csv(
    "../output_reprogramming/spearman_for_each_gene_example.csv"
)

### Computing Spearman correlation for each subprocess

In [9]:
def compute_subprocess_spearman(
    metadata: pd.DataFrame,
    expr_data: pd.DataFrame,
    all_hallmarks: pd.DataFrame,
    fdr_alpha: float = 0.05,
    min_genes: int = 3,
) -> pd.DataFrame:
    """
    Computes aggregated Spearman correlation for each Subprocess within Hallmark gene sets.

    For each gene in `all_hallmarks` present in `expr_data`, computes
    Spearman’s rho and p-value between expression vectors and reprogramming days.
    Then performs FDR correction, merges the results with annotations, and aggregates by
    unique (Subprocess, Hallmark) pairs: median rho, median p-value, median FDR,
    total number of genes, and number of significant genes.

    Parameters
    ----------
    metadata : pd.DataFrame
        Metadata DataFrame; must contain a 'days_of_reprogramming'
        column (numeric or coercible to numeric).
    expr_data : pd.DataFrame
        Expression data matrix (rows are samples, columns are gene Ensembl IDs).
    all_hallmarks : pd.DataFrame
        Hallmark annotation table with columns
        'ENSEMBL', 'Subprocess', and 'Hallmark'.
    fdr_alpha : float, optional
        Significance threshold for FDR correction (default=0.05).
    min_genes : int, optional
        Minimum number of genes in a Subprocess required to compute metrics
        (default=3).

    Returns
    -------
    pd.DataFrame
        DataFrame with columns:
        ['Subprocess', 'Hallmark', 'MedianSpearmanR', 'MedianPValue',
         'MedianFDR', 'N_Genes', 'N_Significant'],
        sorted by ['Hallmark', 'Subprocess'].
    """

    time_vec = pd.to_numeric(metadata["days_of_reprogramming"], errors="coerce")

    valid = (
        all_hallmarks[all_hallmarks["ENSEMBL"].isin(expr_data.columns)]
        .dropna(subset=["Subprocess"])
        .drop_duplicates(subset=["ENSEMBL", "Subprocess", "Hallmark"])
    )

    records = []
    for ensembl_id in valid["ENSEMBL"].unique():
        expr = expr_data[ensembl_id]
        if expr.nunique() <= 1:
            continue
        rho, pval = spearmanr(expr, time_vec, nan_policy="omit")
        records.append({"Gene": ensembl_id, "SpearmanR": rho, "p_value": pval})

    df_genes = pd.DataFrame(records)
    if df_genes.empty:
        return pd.DataFrame(
            columns=[
                "Subprocess",
                "Hallmark",
                "MedianSpearmanR",
                "MedianPValue",
                "MedianFDR",
                "N_Genes",
                "N_Significant",
            ]
        )

    df_genes["FDR"] = multipletests(df_genes["p_value"], method="fdr_bh")[1]
    df_genes["significant"] = df_genes["FDR"] < fdr_alpha

    annot = valid[["ENSEMBL", "Subprocess", "Hallmark"]].rename(
        columns={"ENSEMBL": "Gene"}
    )
    df_genes = df_genes.merge(annot, on="Gene", how="left")

    subprocess_stats = []
    groups = df_genes[["Subprocess", "Hallmark"]].drop_duplicates()
    for _, row in groups.iterrows():
        sub = row["Subprocess"]
        hm = row["Hallmark"]
        subset = df_genes[
            (df_genes["Subprocess"] == sub) & (df_genes["Hallmark"] == hm)
        ]
        n = len(subset)
        if n >= min_genes:
            median_rho = subset["SpearmanR"].median()
            median_p = subset["p_value"].median()
            median_fdr = subset["FDR"].median()
            n_sig = subset["significant"].sum()
        else:
            median_rho = np.nan
            median_p = np.nan
            median_fdr = np.nan
            n_sig = np.nan

        subprocess_stats.append(
            {
                "Subprocess": sub,
                "Hallmark": hm,
                "MedianSpearmanR": median_rho,
                "MedianPValue": median_p,
                "MedianFDR": median_fdr,
                "N_Genes": n,
                "N_Significant": n_sig,
            }
        )

    result = (
        pd.DataFrame(subprocess_stats)
        .sort_values(["Hallmark", "Subprocess"])
        .reset_index(drop=True)
    )
    return result

In [10]:
spearman_for_each_subprocess = compute_subprocess_spearman(
    metadata, data, all_hallmarks
)

In [11]:
spearman_for_each_subprocess.head()

,Subprocess,Hallmark,MedianSpearmanR,MedianPValue,MedianFDR,N_Genes,N_Significant
0,Autophagy core,Autophagy,-0.018688,0.022570,0.047308,175,88.0
1,Autophagy downregulators,Autophagy,-0.150753,0.010337,0.023988,8,5.0
2,Autophagy upregulators,Autophagy,-0.010773,0.014189,0.030720,58,30.0
3,Docking and fusion,Autophagy,-0.069617,0.079822,0.138174,20,8.0
4,Lysosome,Autophagy,-0.045264,0.028676,0.058101,149,73.0


In [12]:
spearman_for_each_subprocess.to_csv(
    "../output_reprogramming/spearman_for_each_subprocess.csv"
)